# 📘 5_demo_pipeline.ipynb

- **Objectif** : Démonstration du pipeline complet MLflow (dataset → préparation → entraînement → logging → prédiction)

---


In [ ]:
import mlflow
import mlflow.sklearn
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.preprocessing import StandardScaler

# 🔹 1. Charger le dataset

In [ ]:
# ------------------------------------------------------------
# 🔹 1. Charger le dataset
# ------------------------------------------------------------
df = pd.read_csv("dataset.csv")
df = df.drop(columns=["RowNumber", "CustomerId", "Surname"], errors="ignore")
df = df[df["Age"] <= 80]

## Encodage simplifié

In [ ]:
# Encodage simplifié
df = pd.get_dummies(df, drop_first=True)

In [ ]:
X = df.drop("Exited", axis=1)
y = df["Exited"]

# 🔹 2. Split des données

In [ ]:
# ------------------------------------------------------------
# 🔹 2. Split des données
# ------------------------------------------------------------
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

In [ ]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# 🔹 3. Récupération du meilleur modèle depuis MLflow

In [ ]:
# ------------------------------------------------------------
# 🔹 3. Récupération du meilleur modèle depuis MLflow
# ------------------------------------------------------------
mlflow.set_tracking_uri("http://localhost:5000")
mlflow.set_experiment("Churn_Prediction")

In [ ]:
client = mlflow.tracking.MlflowClient()
runs = client.search_runs(
    experiment_ids=["0"], order_by=["metrics.f1 DESC"], max_results=1
)
best_run = runs[0]
best_model_uri = f"runs:/{best_run.info.run_id}/model"

In [ ]:
print(f"✅ Meilleur modèle chargé depuis MLflow : {best_model_uri}")

In [ ]:
model = mlflow.sklearn.load_model(best_model_uri)

# 🔹 4. Prédiction sur les données test

In [ ]:
# ------------------------------------------------------------
# 🔹 4. Prédiction sur les données test
# ------------------------------------------------------------
y_pred = model.predict(X_test_scaled)

# 🔹 5. Rapport de performance

In [ ]:
# ------------------------------------------------------------
# 🔹 5. Rapport de performance
# ------------------------------------------------------------
print("\n📊 Rapport de classification :\n")
print(classification_report(y_test, y_pred))

# 🔹 6. Exemple de prédiction individuelle

In [ ]:
# ------------------------------------------------------------
# 🔹 6. Exemple de prédiction individuelle
# ------------------------------------------------------------
sample = X_test.iloc[0:1]
sample_scaled = scaler.transform(sample)
pred_sample = model.predict(sample_scaled)
print(f"🔍 Exemple de prédiction sur un client : {pred_sample[0]}")